<a href="https://colab.research.google.com/github/ujjalkumarmaity/research-paper-implementation/blob/main/Learning-Text-Similarity-with-Siamese-Recurrent-Networks/Siamese%20Recurrent%20Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Learning Text Similarity with Siamese Recurrent Networks

https://aclanthology.org/W16-1617.pdf

<!-- ![image.png](attachment:image.png) -->
<img src = 'download.png'>

In [1]:
import pandas as pd
import numpy as np
import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Model
from keras import layers
from keras import backend as K
import random
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
import tensorflow as tf

In [2]:
def prepare_dataset(path):
    df = pd.read_csv(path, sep='\t', lineterminator='\n',header=None)
    df = df.rename(columns = {0:'name1',1:'name2',3:'label'})
    x1 = []
    x2 = []
    label = []
    name1 = df.name1.values.tolist()
    name2 = df.name2.values.tolist()
    for n1,n2 in zip(name1,name2):
        if random.random()>0.5:
            x1.append(n1)
            x2.append(n2)
        else:
            x1.append(n2)
            x2.append(n1)
        label.append(1)
    all_name = np.asarray(name1+name2)
    shuffle_name = all_name[np.random.permutation(np.arange(len(all_name)))]
    for n1,n2 in zip(all_name,shuffle_name):
        if random.random()>0.5:
            x1.append(n1)
            x2.append(n2)
        else:
            x1.append(n2)
            x2.append(n1)
        label.append(0)
    return pd.DataFrame({'name1':x1,'name2':x2,'label':label})

In [3]:
url = 'https://raw.githubusercontent.com/ujjalkumarmaity/research-paper-implementation/main/Learning-Text-Similarity-with-Siamese-Recurrent-Networks/person_match.train2'
df = prepare_dataset(url)

In [4]:
df.head(2)

,name1,name2,label
0,Fedorowicz,Taras Fedorovych,1
1,Rodger,"Alan Rodger, Baron Rodger of Earlsferry",1


In [5]:
MAX_SEQ_LEN = 70


In [6]:
def prepere_training_data(df,tokenizer):
    name1_seq = tokenizer.texts_to_sequences(df['name1'])
    name2_seq = tokenizer.texts_to_sequences(df['name2'])

    name1_seq = pad_sequences(name1_seq,maxlen=MAX_SEQ_LEN)
    name2_seq = pad_sequences(name2_seq,maxlen=MAX_SEQ_LEN)
    return name1_seq,name2_seq,df['label'].values

tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(df['name1'])
train,test = train_test_split(df,test_size=0.2,stratify = df['label'])
train_text2seq_1,train_text2seq_2,train_label = prepere_training_data(train,tokenizer)
test_text2seq_1,test_text2seq_2,test_label = prepere_training_data(test,tokenizer)


## Model

In [7]:

num_word = len(tokenizer.word_index)+1
def euclidean_distance(vects):
    x, y = vects
    sum_square = tf.math.reduce_sum(tf.math.square(x - y), axis=1, keepdims=True)
    return tf.math.sqrt(tf.math.maximum(sum_square, tf.keras.backend.epsilon()))

input_1 = layers.Input((28, 28, 1))
input_2 = layers.Input((28, 28, 1))

inp_seq = layers.Input(shape=(MAX_SEQ_LEN,))
x = layers.Embedding(num_word,output_dim=16,mask_zero=False)(inp_seq)
x = layers.BatchNormalization()(x)
x = layers.Bidirectional(layers.LSTM(64,return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64,return_sequences=True))(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(128)(x)

embed_network = keras.Model(inp_seq, x)

inp_seq1 = layers.Input(shape=(MAX_SEQ_LEN,))
inp_seq2 = layers.Input(shape=(MAX_SEQ_LEN,))

network1 = embed_network(inp_seq1)
network2 = embed_network(inp_seq2)


merge = layers.Lambda(euclidean_distance)([network1, network2])
merge = layers.BatchNormalization()(merge)
out = layers.Dense(1,activation='sigmoid')(merge)

model = Model(inputs=[inp_seq1,inp_seq2],outputs = out)
opt = SGD(lr=0.01)
# model.compile(optimizer=opt,loss='binary_crossentropy',metrics='acc')
# model.fit([train_text2seq_1,train_text2seq_2,],train_label,epochs=5,batch_size=64,verbose=1,
#           validation_data=([test_text2seq_1,test_text2seq_2],test_label))

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [8]:
def loss(margin=1):
    """Provides 'contrastive_loss' an enclosing scope with variable 'margin'.

    Arguments:
        margin: Integer, defines the baseline for distance for which pairs
                should be classified as dissimilar. - (default is 1).

    Returns:
        'contrastive_loss' function with data ('margin') attached.
    """

    # Contrastive loss = mean( (1-true_value) * square(prediction) +
    #                         true_value * square( max(margin-prediction, 0) ))
    def contrastive_loss(y_true, y_pred):
        """Calculates the contrastive loss.

        Arguments:
            y_true: List of labels, each label is of type float32.
            y_pred: List of predictions of same length as of y_true,
                    each label is of type float32.

        Returns:
            A tensor containing contrastive loss as floating point value.
        """
        y_pred = tf.cast(y_pred, tf.float32)
        y_true = tf.cast(y_true, tf.float32)


        square_pred = tf.math.square(y_pred)
        margin_square = tf.math.square(tf.math.maximum(margin - (y_pred), 0))
        return tf.math.reduce_mean(
            (1 - y_true) * square_pred + (y_true) * margin_square
        )

    return contrastive_loss


In [9]:
margin =1
model.compile(loss=loss(margin=margin), optimizer="adam", metrics=["accuracy"])#RMSprop
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 70)]         0           []                               
                                                                                                  
 input_5 (InputLayer)           [(None, 70)]         0           []                               
                                                                                                  
 model (Functional)             (None, 128)          158096      ['input_4[0][0]',                
                                                                  'input_5[0][0]']                
                                                                                                  
 lambda (Lambda)                (None, 1)            0           ['model[0][0]',            

In [10]:
model.fit([train_text2seq_1,train_text2seq_2,],train_label,epochs=5,batch_size=16,verbose=1,
          validation_data=([test_text2seq_1,test_text2seq_2],test_label))


Epoch 1/5
1980/1980 [==============================] - 643s 316ms/step - loss: 0.1414 - accuracy: 0.8017 - val_loss: 0.1070 - val_accuracy: 0.8564
Epoch 2/5
1980/1980 [==============================] - 623s 315ms/step - loss: 0.1036 - accuracy: 0.8629 - val_loss: 0.0932 - val_accuracy: 0.8807
Epoch 3/5
1980/1980 [==============================] - 599s 303ms/step - loss: 0.0906 - accuracy: 0.8813 - val_loss: 0.0838 - val_accuracy: 0.8910
Epoch 4/5
1980/1980 [==============================] - 579s 292ms/step - loss: 0.0816 - accuracy: 0.8953 - val_loss: 0.0788 - val_accuracy: 0.8975
Epoch 5/5
1980/1980 [==============================] - 577s 291ms/step - loss: 0.0724 - accuracy: 0.9083 - val_loss: 0.0790 - val_accuracy: 0.8970
